MAKE A BACKUP / COPY OF THIS WHENEVER UPDATING. 

This colab file is shared with multiple people, so anything you do on here may be overwritten (colab does not work in the same way as google docs, and it is easy to overwrite someone else's save). Creating a backup each time you make an important update will prevent your work from being deleted. Working on this file is fine. To prevent overwriting, make sure to use your own cell

This will contain the newest version of underlords AI. Info can be posted here on what to work on. The old code (which we will need to revamp) is on the github, which can be found on the discord.

# Simulator Class 

The following JSON file contains info about the Underlords pieces which will be used in making the simulator

https://github.com/SteamDatabase/GameTracking-Underlords/blob/master/game/dac/pak01_dir/scripts/units.json

Load the contents of the JSON 

In [10]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
import json

file_path = 'units.json'
url = 'https://raw.githubusercontent.com/SteamDatabase/GameTracking-Underlords/master/game/dac/pak01_dir/scripts/units.json'

In [0]:
# Downloads a json frile from an URL and save to file_path
def downloads_json(url, file_path):
  import requests
  myfile = requests.get(url, allow_redirects=True)
  open(file_path, 'wb').write(myfile.content)

# Load json data to a python dictionary from file_path
def load_json(file_path):  
  import json 
  return json.load(open(file_path))
  

# Fetch data: download then load a json to a dictionary
def fetch_data(url, file_path='sample.json'):
  downloads_json(url, file_path)
  return load_json(file_path)


In [0]:
unit_info = fetch_data(url, file_path)

In [14]:
print(unit_info)

{'abaddon': {'abilities': ['abaddon_aphotic_shield'], 'armor': 10, 'attackAnimationPoint': 0.56, 'attackRange': 1, 'attackRate': 1.5, 'damageMin': [45, 90, 216], 'damageMax': [55, 110, 264], 'displayName': '#dac_hero_name_abaddon', 'dota_unit_name': 'npc_dota_hero_abaddon', 'draftTier': 3, 'goldCost': 3, 'health': [1500, 3000, 6000], 'id': 32, 'hintWords': ['#dac_hero_hint_melee', '#dac_hero_hint_explosive_shield'], 'keywords': 'undead knight', 'magicResist': 0, 'maxmana': 100, 'model': 'models/heroes/abaddon/abaddon.vmdl', 'model_scale': 0.6, 'texturename': 'abaddon', 'soundSet': 'Hero_Abaddon', 'movespeed': 325, 'healthBarOffset': -45}, 'alchemist': {'abilities': ['alchemist_acid_spray'], 'armor': 5, 'attackAnimationPoint': 0.35, 'attackRange': 1, 'attackRate': 1, 'damageMin': [50, 100, 200], 'damageMax': [70, 140, 280], 'displayName': '#dac_hero_name_alchemist', 'dota_unit_name': 'npc_dota_hero_alchemist', 'draftTier': 4, 'goldCost': 4, 'health': [2000, 4000, 8000], 'id': 46, 'hintW

First, we should create a class which makes it easy for us to access important information about each of the characters. 

In [0]:
class UnitInfo:
  def __init__(self):
    self.unit_info = unit_info

  # Return a list of synergies for a certain uit
  def get_synergies(self, unit):
    return self.unit_info[unit]['keywords'].split()

  # cost in gold of a unit
  def get_cost(self, unit):
    return self.unit_info[unit]['goldCost']

In [20]:
info = UnitInfo()
info.get_synergies('dragon_knight')
info.get_cost('dragon_knight')

4

We should also create a game simulator class. For now, it will be really basic (so we can get a working prototype of the AI). 

TO DO: 

find probabilities of each unit appearing given player level

In [0]:
# This code cell is for creating a dictionary which assigns an alliance 
# to the number of different units needed for that alliance
# e.g Demon Hunter = 1, Knight = 2, Assassin = 3
from collections import defaultdict

alliance_levels = {
    "assassin": {"boost":3,"max":6},
    "bloodbound": {"boost":2, "max":2},
    "brawny": {"boost":2, "max":4},
    "brute": {"boost":2,"max":4},
    "champion": {"boost":1, "max":1},
    "deadeye": {"boost":2, "max":2},
    "demon": {"boost":1, "max":1},
    "dragon": {"boost":2, "max":2},
    "druid": {"boost":2, "max":4},
    "healer": {"boost":3, "max":3},
    "heartless": {"boost":2, "max":6},
    "human": {"boost":2, "max":6},
    "hunter": {"boost":3,"max":6},
    "insect": {"boost":2, "max":4},
    "inventor": {"boost":2, "max":4},
    "knight": {"boost":2, "max":6},
    "mage": {"boost":3,"max":6},
    "primordial": {"boost":2, "max":4},
    "savage": {"boost":2, "max":6},
    "scaled": {"boost":2, "max":4}, 
    "scrappy": {"boost":2, "max":6},
    "shaman": {"boost":2, "max":4}, 
    "troll": {"boost":2, "max":4},
    "warlock": {"boost":2, "max":6},
    "warrior": {"boost":3,"max":6}
}

In [28]:
# probability of tiers given level
level_prob = {
        '1' : [1, 0, 0, 0, 0],
        '2' : [0.7, 0.3, 0, 0, 0],
        '3' : [0.6, 0.35, 0.05, 0, 0],
        '4' : [0.5, 0.35, 0.15, 0, 0], 
        '5' : [	0.4, 0.35,	0.25,	0,	0], 
        '6' : [0.35, 0.30, 0.3, 0.05], 
        '7' : [0.25, 0.3, 0.35, 0.1, 0], 
        '8' : [0.22, 0.27, 0.35, 0.15, 0.1], 
        '9' : [0.2, 0.25, 0.3, 0.2, 0.3], 
        '10' : [0.15, 0.21, 0.28, 0.3, 0.06]
    }

# Create a dictionary which maps the cost / draft tier to a list of heroes 
# which are members of that tier
# e.g 4 = ['Dragon Knight', 'Kunkka', ...]
print(info.get_cost("dragon_knight"))
tier_to_unit = defaultdict(list)
for unit in unit_info:
  try:
    cost = info.get_cost(unit)
  except:
    print(unit)
  if cost != -1 and cost != 0:
    tier_to_unit[str(cost)].append(unit)

class SimpleSimulator():
  def __init__(self):
    self.inventory = defaultdict(int)
    self.player_level = 1
    self.gold = 5
    self.xp = 1
  
  def reset(self):
    self.inventory = defaultdict(int)
    self.player_level = 1
    self.gold = 5
    self.xp = 1

  # WORKS AND IS DONE
  def generate_units(self, player_level): # player_level affects probability
    
    # get probability distribution of tiers based on player level
    probabilities = level_prob[str(player_level)]
    units = []

    from numpy.random import choice
    # get units of 5 slots which show up in store
    for i in range(5):
      # choose draft tier of slot i
      draw = choice([1, 2, 3, 4, 5], 1, p=probabilities)
      # choose a hero at random from the pool (account for pool size)
      unit = choice(tier_to_unit[str(draw[0])])
      units.append(unit) # add to the 5 slots

    return units # list of units in the shop
  def calcLevel(self,count):
    l1 = 0
    l2 = 0
    l3 = 0
    if(count/9>1):
      # Level 3's
      l3 = math.floor(count/9)
      count -= 9*(math.floor(count/9))
    if(count/3>1):
      # Level 2's
      l2 = math.floor(count/3)
      count -= 3*(math.floor(count/3))
    
    # Level 1's
    l1 = count
    return ([l1,l2,l3])
  # Given the amount of spaces on the board, this function will find all 
  # combinations of pieces, and will choose the one with the maximum reward
  def calculate_reward(self):
    fullInventory=[]
    unitLevels={}
    for unit in self.inventory:
      unitCount = self.inventory[unit]
      unitLevels[unit] = calcLevel(unitCount)

    for unit in unitLevels:
      for index,level in enumerate(unitLevels[unit]):
        fullInventory.extend([(unit+str(index+1))]*level)

    # defining parallel array to all combos
    allComboSynergyLevel = []
    for combo in itertools.combinations(fullInventory,self.player_level):
      comboSynergy = defaultdict(int)

      # Goes through each synergy of each unit and tallies the frequency
      for unit in combo:
        unit = unit[:-1]
        synergies = info.get_synergies(unit)
        for synergy in synergies:
          comboSynergy[synergy] +=1

      # Goes through the frequencies of each synergy and calculates level and adds it to parallel array
      # holds levels for each synergy
      temp ={}
      for synergy in comboSynergy:
        boost=alliance_levels[synergy]["boost"]
        cap = alliance_levels[synergy]["max"]
        synergyLevel = comboSynergy[synergy]/boost
        if(synergyLevel <1):
          temp[synergy]=0
        elif(synergyLevel > cap/boost):
          temp[synergy] = cap
        else:
          temp[synergy] = math.floor(synergyLevel)
      
      allComboSynergyLevel.append(temp)
    return 0

  # Returns observation space of network
  def get_obs_space(self, units):
    # Instead of making the observation space the entire set of units,
    # we can make it related to just the units that are in the store
    obs_space = []
    for unit in units:
      obs_space.append(self.inventory[unit])
    return obs_space

  # actions - what to do 
  def round(self, units, actions, rnd):
    reward_before = self.calculate_reward()
    current_state = self.get_obs_space(units)
    
    for action in actions:
      # if a unit is being bought, add it to the inventory
      if action[:3] == 'buy':
        self.inventory[action[3:]] += 1
      elif action[:4] == 'sell':
        # Here, we account for the fact that technically 3 units are being sold
        # if a level 2 unit is sold
        # However,  if there are 4 pieces, that means there is 1 level 2 and 1 
        # level 1, so we make the assumption that the level 1 is sold
        if self.inventory[action[4:]] % 3 == 0 and self.inventory[action[4:]] != 9:
          self.inventory[action[4:]] -= 3
          self.gold += (3 * info.get_cost(action[4:]))
        # if the unit is level 9, sell all of them
        elif self.inventory[action[4:]] == 9:
          self.inventory[action[4:]] -= 9
          self.gold += (9 * info.get_cost(action[4:]))
        else:
          self.inventory[action[4:]] -= 1
          self.gold += info.get_cost(action[4:])
      # invariant -> enough gold
      elif action == 'level':
        while self.xp <= (2 ** self.level):
          self.xp += 4
          self.gold -= 5
        self.player_level += 1
        # if 39/40 and level up, then the player is level 11 with 3 leftover xp
        self.xp = self.xp % (2 ** self.level)


    self.xp += 1
    if self.xp == 2 ** self.player_level:
      self.player_level += 1
      self.xp = 0 
    self.gold += (5 + self.gold * 0.1) 
    
    reward_after = self.calculate_reward()
    reward = reward_after - reward_before
    
    current_new_state = self.get_obs_space(units)

    return (current_state, actions, reward, current_new_state, rnd == 35)
  
  

4
neutral_melee
neutral_melee_mega
neutral_ranged
neutral_golem_a
neutral_golem_b
neutral_wolf_big
neutral_wolf_small
neutral_bear_a
neutral_bear_b
neutral_vulture_a
neutral_vulture_b
neutral_thunder_lizard_big
neutral_thunder_lizard_small
neutral_black_dragon
neutral_troll_dark_a
neutral_troll_dark_b
neutral_troll_dark_frost
neutral_nian
neutral_roshan
anessix
hobgen
horn_of_the_alpha_thunderhide
anessix_golem


In [27]:
# tier_to_unit
test = SimpleSimulator()

defaultdict(list,
            {'1': ['antimage',
              'axe',
              'bat_rider',
              'bounty_hunter',
              'drow_ranger',
              'enchantress',
              'razor',
              'shadow_shaman',
              'tiny',
              'tusk',
              'venomancer',
              'warlock',
              'bloodseeker',
              'nyx_assassin',
              'shadow_demon',
              'snapfire'],
             '2': ['beastmaster',
              'chaos_knight',
              'crystal_maiden',
              'furion',
              'luna',
              'ogre_magi',
              'puck',
              'queen_of_pain',
              'slardar',
              'timbersaw',
              'wind_ranger',
              'witch_doctor',
              'pudge',
              'weaver',
              'magnus',
              'dazzle',
              'storm_spirit',
              'ember_spirit'],
             '3': ['abaddon',
              'clockwerk',
 